# <p align="center">Siemens Sales Forecast</p>

---

## <p align="center">*2 - Feature Selection & Modeling*</p>

---

### 👥 **Team Members**
- **Ana Farinha** *(Student Number: 20211514)*  
- **António Oliveira** *(Student Number: 20211595)*  
- **Mariana Neto** *(Student Number: 20211527)*  
- **Salvador Domingues** *(Student Number: 20240597)*  

📅 **Date:** *April 1, 2025*  
📍 **Prepared for:** *Siemens*  

**GitHub Repo:** https://github.com/MGN19/Siemens-forecast

---

# ToC

<a class="anchor" id="top"></a>


1. [Import Libraries & Data](#1.-Import-Libraries-&-Data) <br><br>

2. [Product Category #1](#Product-Category-#1) <br><br>

In [1]:
## CELL TYPES (remover depois)

<div class="alert-danger">
    
test

<div class="alert-warning">
    
test

<div class="alert-info">
    
test

# 1. Import Libraries & Data

In [1]:
import os
import pandas as pd

pd.set_option('display.max_columns', None)

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")

import fs_modelling as m

**Data**

In [14]:
X_train = pd.read_csv('./data/X_train_data/X_train.csv', index_col = 'Unnamed: 0')
X_val = pd.read_csv('./data/X_val_data/X_val.csv', index_col = 'Unnamed: 0')

def import_all_csvs_as_vars(folder):
    for file in os.listdir(folder):
        if file.endswith('.csv'):
            df_name = file.replace('.csv', '')
            df = pd.read_csv(os.path.join(folder, file))
            globals()[df_name] = df
            print(f"Loaded {df_name}")

# Import each CSV file as individual DataFrames
import_all_csvs_as_vars('data/y_train_data')
import_all_csvs_as_vars('data/y_val_data')

Loaded y_train_36
Loaded y_train_8
Loaded y_train_20
Loaded y_train_9
Loaded y_train_4
Loaded y_train_11
Loaded y_train_5
Loaded y_train_12
Loaded y_train_13
Loaded y_train_6
Loaded y_train_16
Loaded y_train_3
Loaded y_train_1
Loaded y_train_14
Loaded y_val_1
Loaded y_val_3
Loaded y_val_6
Loaded y_val_5
Loaded y_val_4
Loaded y_val_16
Loaded y_val_14
Loaded y_val_11
Loaded y_val_13
Loaded y_val_12
Loaded y_val_36
Loaded y_val_20
Loaded y_val_9
Loaded y_val_8


# Product Category #1

<a href="#top">Top &#129033;</a>

In [11]:
datasets = [X_train, X_val, y_train_1, y_val_1]

**Scaling**

In [15]:
X_train_scaled, X_val_scaled = m.scale_data(X_train, 
                                          X_val, 
                                          scaler_type='minmax')

## 2.1 Feature Selection

In [17]:
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectFdr, f_regression, RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, LassoCV
from xgboost import XGBRegressor

def feature_selection(X_train, y_train, method='all',
                      rfe_model=None, corr_threshold=0.85, 
                      importance_threshold='mean', lasso_cv=True):
    selected_features = []

    if method == 'correlation':
        corr_matrix = X_train.corr()
        upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        correlated_features = [column for column in upper_triangle.columns if any(upper_triangle[column].abs() > corr_threshold)]
        selected_features = [col for col in X_train.columns if col not in correlated_features]
        print(f'Selected {len(selected_features)} features by correlation')
        
    elif method == 'rfe':
        if rfe_model is None:
            rfe_model = LinearRegression()
            
        rfecv = RFECV(estimator=rfe_model, cv=5, scoring='neg_root_mean_squared_error')
        rfecv.fit(X_train, y_train)
        selected_features = X_train.columns[rfecv.support_]
        print(f'Selected {len(selected_features)} features by RFECV')

    elif method == 'importance':
        model = RandomForestRegressor()
        model.fit(X_train, y_train)
        importances = model.feature_importances_

        if importance_threshold == 'mean':
            threshold_value = np.mean(importances)
        elif importance_threshold == 'median':
            threshold_value = np.median(importances)
        else:
            threshold_value = float(importance_threshold)

        selected_features = X_train.columns[importances > threshold_value]
        print(f'Selected {len(selected_features)} features by importance with threshold {threshold_value}')

    elif method == 'lasso':
        lasso = LassoCV(cv=5, random_state=42).fit(X_train, y_train)
        selected_features = X_train.columns[lasso.coef_ != 0].tolist()
        print(f'Selected {len(selected_features)} features by Lasso regularization')

    elif method == 'all':
        corr_features = feature_selection(X_train, y_train, method='correlation', corr_threshold=corr_threshold)
        rfe_features = feature_selection(X_train, y_train, method='rfe', rfe_model=rfe_model)
        importance_features = feature_selection(X_train, y_train, method='importance', importance_threshold=importance_threshold)
        lasso_features = feature_selection(X_train, y_train, method='lasso')

        selected_features = list(set(corr_features) & set(rfe_features) & set(importance_features) & set(lasso_features))
        print(f'Selected {len(selected_features)} features that intersect across all methods')

    return selected_features


In [18]:
selected_features = feature_selection(X_train_scaled, y_train_1, 
                                      method='all', 
                                      rfe_model = XGBRegressor(), 
                                      corr_threshold=0.85, 
                                      importance_threshold = 'mean')

selected_features

Selected 41 features by correlation
Selected 3 features by RFECV
Selected 27 features by importance with threshold 0.008130081300813009


ValueError: Input X contains NaN.
LassoCV does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [102]:
import pandas as pd
import numpy as np
import os
from lazypredict.Supervised import LazyRegressor
from sklearn.metrics import mean_squared_error
from prophet import Prophet  # Prophet model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA  # ARIMA model

def modelling(X_train, y_train, X_test, y_test, 
              features_used, 
              metric='RMSE', 
              model_choice='arima', 
              save_path='./modelling_csvs/results.csv'):


    # If ARIMA is chosen
    if model_choice == 'arima':
        if len(features_used) > 1:
            raise ValueError('ARIMA only accepts 1 feature for the target variable.')
        model = ARIMA(y_train, order=(1, 1, 1))
        model_fit = model.fit()

        # Predict
        predictions = model_fit.forecast(len(y_test))
        
        # Calculate RMSE or any other metric
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        best_model_name = 'ARIMA'
        best_score = rmse
        print(f'ARIMA RMSE: {rmse}')

    # Initialize LazyRegressor if 'lazy' model is chosen
    elif model_choice == 'lazy':
        regressor = LazyRegressor(verbose=0)
        models, predictions = regressor.fit(X_train[features_used], X_test[features_used], y_train, y_test)
        
        # Select the best model based on the metric
        best_model = models.sort_values(by=metric).iloc[0]
        best_model_name = best_model.name
        best_score = best_model[metric]
        print(f'Best model: {best_model_name}')
        print(f'{metric} of the best model: {best_score}')
    
    # If Prophet is chosen
    elif model_choice == 'prophet':
        raise ValueError('NOT WORKING YET')
        # Initialize an empty DataFrame to hold future feature values
        future_features = pd.DataFrame()

        # Iterate through each column in 'features_used'
        for column in features_used:
            # Isolate the current column into a new DataFrame 'df1'
            df1 = X_train[[column]].copy()

            # Reset the index of 'df1' and rename columns to fit Prophet's expected format
            data = (df1.reset_index()
                    .rename(columns={'index': 'ds', f'{column}': 'y'}))

            # Initialize Prophet model
            model = Prophet()

            # Fit the model to the data
            model.fit(data)

            # Create a DataFrame representing future dates to make predictions
            future = model.make_future_dataframe(periods=10, freq='MS')

            # Forecast future dates
            forecast_index = model.predict(future)

            forecast_index = forecast_index[['ds', 'yhat']]
            
            # Set the date column as the index
            forecast_index = forecast_index.set_index('ds')

            # Add the forecasted values to the 'future_features' DataFrame
            future_features[column] = forecast_index['yhat'].values

        # Reset the index to use date as a regular column
        future_features.reset_index(inplace=True)

        # Add the date column to 'future_features'
        future_features['ds'] = forecast_index['ds'].values

        # Set date as the index of 'future_features'
        future_features.set_index('ds', inplace=True)

        # For demonstration, using RMSE here:
        predicted_values = future_features[features_used].mean(axis=1)  # For simplicity, take the mean of all predictions
        rmse = mean_squared_error(y_test, predicted_values, squared=False)
        
        # Compare RMSE to get best model
        if rmse < best_score:
            best_score = rmse
            best_model_name = 'prophet'
            
    # Prepare row to save
    result_row = {
        'Features Used': ', '.join(features_used),
        'Best Model': best_model_name,
        metric: best_score
    }
    
    # Check if file exists; if not, create with headers
    if not os.path.isfile(save_path):
        results_df = pd.DataFrame([result_row])
        results_df.to_csv(save_path, index=False)
    else:
        # Append without overwriting
        results_df = pd.DataFrame([result_row])
        results_df.to_csv(save_path, mode='a', header=False, index=False)
    
    return best_model_name, best_score


In [103]:
best_model_name, best_score  = modelling(X_train_1_scaled, y_train_1, X_val_1_scaled, y_val_1, 
              features_used=selected_features, 
              metric='RMSE', 
              model_choice='prophet')

ValueError: NOT WORKING YET

In [46]:
from prophet import Prophet
import pandas as pd
from sklearn.metrics import mean_squared_error

def prophet_forecast(X_train, y_train, features_used, periods=10, freq='MS'):
    """
    Function to perform forecasting using Prophet for each feature in 'features_used'.

    Parameters:
    - X_train: DataFrame containing the training data
    - y_train: Actual ground truth values for RMSE calculation (training data)
    - features_used: List of features/columns to forecast using Prophet
    - periods: Number of future periods to predict (default=10)
    - freq: Frequency of the periods (default='MS' for monthly start)

    Returns:
    - best_model_name: Name of the best model ('prophet')
    - best_score: The lowest RMSE score for the model
    """
    best_score = float('inf')
    best_model_name = None

    # Initialize an empty DataFrame to hold future feature values
    future_features = pd.DataFrame()

    # Iterate through each column in 'features_used'
    for column in features_used:
        # Isolate the current column into a new DataFrame 'df1'
        df1 = X_train[[column]].copy()

        # Reset the index of 'df1' and rename columns to fit Prophet's expected format
        data = (df1.reset_index()
                .rename(columns={'index': 'ds', f'{column}': 'y'}))

        # Initialize Prophet model
        model = Prophet()

        # Fit the model to the data
        model.fit(data)

        # Create a DataFrame representing future dates to make predictions
        future = model.make_future_dataframe(periods=periods, freq=freq)

        # Forecast future dates
        forecast_index = model.predict(future)

        # Select relevant columns ('ds' for date, 'yhat' for predictions)
        forecast_index = forecast_index[['ds', 'yhat']]

        # Set the date column as the index
        forecast_index = forecast_index.set_index('ds')

        # Add the forecasted values to the 'future_features' DataFrame
        future_features[column] = forecast_index['yhat'].values

    # Reset the index of the future_features DataFrame to use 'ds' as a regular column
    future_features.reset_index(inplace=True)

    # Add the date column to 'future_features'
    future_features['ds'] = forecast_index.index.values

    # Set 'ds' as the index of 'future_features'
    future_features.set_index('ds', inplace=True)

    # Ensure we only compare the forecasted values against a valid subset of y_train
    # For simplicity, we will compare the mean of the forecasted values to the corresponding `y_train` values
    predicted_values = future_features[features_used].mean(axis=1)  # For simplicity, take the mean of all predictions

    rmse = mean_squared_error(y_train, predicted_values)

    # Compare RMSE to get the best model
    if rmse < best_score:
        best_score = rmse
        best_model_name = 'prophet'

    return best_model_name, best_score


In [47]:
prophet_forecast(X_train_1_scaled, y_train_1, selected_features) 

11:44:03 - cmdstanpy - INFO - Chain [1] start processing
11:44:03 - cmdstanpy - INFO - Chain [1] done processing
11:44:03 - cmdstanpy - INFO - Chain [1] start processing
11:44:03 - cmdstanpy - INFO - Chain [1] done processing


ValueError: Found input variables with inconsistent numbers of samples: [31, 41]

In [34]:
!pip install statsmodels

  Using cached statsmodels-0.14.4-cp313-cp313-macosx_10_13_x86_64.whl.metadata (9.2 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached statsmodels-0.14.4-cp313-cp313-macosx_10_13_x86_64.whl (10.2 MB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)


In [42]:
X_train_1_selected = X_train_1_scaled[selected_features]
X_val_1_selected = X_val_1_scaled[selected_features]

best_model_name, best_score = modelling(
    X_train_1_selected, y_train_1, X_val_1_selected, y_val_1, 
    features_used=selected_features, metric='RMSE'
)

100%|██████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 34.63it/s]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 31, number of used features: 0
[LightGBM] [Info] Start training from score 36289610.258065
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet t

In [67]:
# import lazypredict
# from lazypredict.Supervised import LazyRegressor
# from sklearn.metrics import mean_squared_error
# import numpy as np

# def lazy_regressor(X_train, y_train, X_test, y_test, metric='RMSE'):
    
#     # Initialize LazyRegressor
#     regressor = LazyRegressor(verbose=0)
    
#     # Fit the model
#     models, predictions = regressor.fit(X_train, X_test, y_train, y_test)
    
#     # Select the best model based on the metric (e.g., RMSE)
#     best_model = models.sort_values(by=metric).iloc[0]
    
#     # Get the model name and the best score
#     best_model_name = best_model.name
#     best_score = best_model[metric]
    
#     print(f'Best model: {best_model_name}')
#     print(f'{metric} of the best model: {best_score}')
    
#     # Return the best model and score
#     return best_model_name, best_score


In [68]:
# best_model_name, best_score = lazy_regressor(X_train_1_scaled, y_train_1,
#                                              X_val_1_scaled, y_val_1,
#                                              metric='RMSE')


100%|████████████████████████████████████| 42/42 [00:02<00:00, 17.66it/s]

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 31, number of used features: 0
[LightGBM] [Info] Start training from score 36289610.258065
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet t

In [43]:
file = pd.read_csv('./lazy_regressor_results.csv')
file

,Features Used,Best Model,RMSE
0,"(W) Price of Base Metals, GER Production Index...",SVR,2887703.96
1,USA Shipments Index_Rolling_Mean_3,HuberRegressor,2823366.38
2,"BC_CHI, USA Shipments Index_Rolling_Mean_3, CH...",HuberRegressor,2830173.62
3,"(W) Price of Base Metals, GER Production Index...",SVR,2887703.92
4,"(W) Price of Natural gas index, USA Production...",SVR,2887703.88


In [75]:
from autosklearn.experimental.askl2 import ForecasterAutoregMultiSeries
from sklearn.metrics import mean_squared_error
import numpy as np

def time_series_forecasting(X_train, y_train, X_test, y_test, 
                            forecast_steps=10, metric='neg_root_mean_squared_error'):

    # Initialize ForecasterAutoregMultiSeries
    forecaster = ForecasterAutoregMultiSeries(steps=forecast_steps, metric=metric)
    
    # Fit the forecaster with training data
    forecaster.fit(X_train, y_train)
    
    # Make predictions on the test set
    y_pred = forecaster.predict(X_test)
    
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print(f'Test RMSE: {rmse}')
    
    return forecaster, y_pred, rmse


ModuleNotFoundError: No module named 'autosklearn'

### 2.1.1 Filter Methods

**Variance Threshold**

In [ ]:
X_train_1_scaled.var() 

**Spearman Correlation**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def correlation_matrix(X, cmap='YlOrBr', font_size = 5):
    """
    Input: X (numerical data)
    Output: Correlation Matrix
    """
    
    # Correlation matrix
    corr_matrix = X.corr().abs()

    # Plot Heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr_matrix, annot=True, cmap=cmap, fmt=".2f", annot_kws={"size": font_size})
    plt.title("Feature Correlation Matrix")
    plt.show()

In [ ]:
correlation_matrix(X_train_1_scaled)

In [ ]:
def top_correlated_features(X, top_n=20, cmap='YlOrBr'):
    corr_matrix = X.corr().abs()
    mean_corr = corr_matrix.mean().sort_values(ascending=False)
    top_features = mean_corr.head(top_n).index
    plt.figure(figsize=(10, 8))
    sns.heatmap(X[top_features].corr(), annot=True, cmap=cmap, fmt=".2f", annot_kws={"size": 8})
    plt.title(f"Top {top_n} Most Correlated Features")
    plt.show()


In [ ]:
top_correlated_features(X_train_1_scaled)

### 2.2.2 Wrapper Methods

**RFE**

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

def rfe(X_train, y_train, X_val, y_val, n_features, model=None):
    """
    Input: 
        X_train, y_train, X_val, y_val: training and validation data
        n_features: number of features to use for RFE
        model: chosen regression model 
    Output: selected features for the best model based on RMSE
    """
    
    best_rmse = np.inf  # Start with a very high RMSE (lower is better)
    best_features = []

    results = {}
    
    for feature in n_features:
        
        # Fit RFE
        rfe = RFE(estimator=model, n_features_to_select=feature)
        rfe.fit(X_train, y_train)

        # Get selected features
        selected_features = X_train.columns[rfe.support_]
        
        print('\n-------------TRAIN-------------')

        # Predictions for Train
        y_pred_train = rfe.predict(X_train)
        
        # Metrics for Training (RMSE)
        mse_train = mean_squared_error(y_train, y_pred_train)
        rmse_train = np.sqrt(mse_train)
        
        print(f"RMSE for {feature} features (Train): {rmse_train:.4f}")
        
        print('----------VALIDATION----------')

        # Predictions for Validation
        y_pred_val = rfe.predict(X_val)
        
        # Metrics for Validation (RMSE)
        mse_val = mean_squared_error(y_val, y_pred_val)
        rmse_val = np.sqrt(mse_val)
        
        print(f"RMSE for {feature} features (Validation): {rmse_val:.4f}")
        
        # Best score based on RMSE (lower is better)
        if rmse_val < best_rmse:
            best_rmse = rmse_val
            best_features = selected_features.tolist()  
        
    print('\n----------FINAL----------')
    print(f'Best RMSE: {best_rmse}')
    print(f'Number of Features: {len(best_features)}')
    print(f'Features: {best_features}')
    


In [ ]:
## TEMP
X_train_1_scaled.fillna(0, inplace = True)
X_val_1_scaled.fillna(0, inplace = True)

n_features = np.arange(1, len(X_train_1_scaled.columns) + 1)
model = LinearRegression()

rfe(X_train_1_scaled, y_train_1, X_val_1_scaled, y_val_1, 
                                n_features = n_features, 
                                model = model)

In [ ]:
## TEMP
X_train_1_scaled.fillna(0, inplace = True)
X_val_1_scaled.fillna(0, inplace = True)


from xgboost import XGBRegressor

n_features = np.arange(1, len(X_train_1_scaled.columns) + 1)
model = XGBRegressor(objective='reg:squarederror',
                     random_state = 100)

rfe(X_train_1_scaled, y_train_1, X_val_1_scaled, y_val_1, 
                                n_features = n_features, 
                                model = model)

### 2.2.3 Embedded Methods

**LASSO**

In [ ]:
from sklearn.linear_model import Lasso
import functions as f
def lasso(X, y, alpha = 0.01, color = f.main_color):

    """
    Input: 
        X, y: data
        alpha: parameter for lasso
    Output: Plot, initial and selected features
    """
    
    # Fit
    lasso = Lasso(alpha=alpha)
    lasso.fit(X, y)
    
    # Get Feature Importance
    importance = pd.Series(lasso.coef_, index=X.columns)
    importance.sort_values().plot(kind="barh", color=color)
    non_zero_importance = importance[importance != 0]
    selected_features = non_zero_importance.index

    # Plot
    plt.title("Lasso Feature Importance")
    plt.xlabel("Coefficient Value")
    plt.show()
    
    # Print Results
    print(f"\nInitial Features: {len(X.columns)}\n")
    print(X.columns.tolist())
    print(f"\nDecision for Numerical Features (lasso ≠ 0): {len(selected_features.tolist())}\n")
    print(selected_features.tolist())


In [ ]:
lasso(X_train_1_scaled, y_train_1, alpha = 0.05)

# Model

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

# Fit the model using the training data
model.fit(X_train_1_scaled, y_train_1)

# Make predictions on the validation set
y_pred_val_1 = model.predict(X_val_1_scaled)

# Calculate RMSE for the validation set
rmse_val_1 = np.sqrt(mean_squared_error(y_val_1, y_pred_val_1))
print(f'Root Mean Squared Error on Validation Set: {rmse_val_1}')

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))

plt.plot(y_val_1.index, y_val_1, label='Actual Validation Values', linestyle='-', color='g')
plt.plot(y_val_1.index, y_pred_val_1, label='Predicted Validation Values', linestyle='--', color='g')

plt.plot(y_train_1.index, y_train_1, label='Actual Training Values', linestyle='-', color='b')

plt.xlabel('Date/Time')
plt.ylabel('Sales')
plt.title('XGBoost Forecasting: Actual vs Predicted (Training and Validation)')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
